<a href="https://colab.research.google.com/github/vicpb/treinando-rn-dp-pytorch-alura/blob/main/funcoes_de_perda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Funções de perda

O módulo nn e suas 1001 utilidades, também fornece as implementações das principais funções de perda. Então vamos primeiro importar o torch e o módulo nn

In [ ]:
import torch
from torch import nn

Antes de tudo, vamos conferir qual dispositivo de hardware está disponível para uso.

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

Vamos trabalhar com o dataset de classificação de vinhos.

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html

In [ ]:
from sklearn import datasets

wine = datasets.load_wine()
data = wine.data
target = wine.target

print(data.shape, target.shape)
print(wine.feature_names, wine.target_names)

Vamos instanciar um MLP com uma camada escondida e uma camada de saída.

In [ ]:
class WineClassifier(nn.Module):
  
  def __init__(self, input_size, hidden_size, out_size):
    super(WineClassifier, self).__init__()

    self.hidden = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.out = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax()

  def forward(self, X):

    feature = self.relu(self.hidden(X))
    output = self.softmax(self.out(feature))

    return output
  
input_size   = data.shape[1]
hidden_size  = 32
out_size     = len(wine.target_names)

net = WineClassifier(input_size, hidden_size, out_size).to(device) #cast to GPU


In [ ]:
print(net)

#Classificação

O primeiro passo é instanciar a função de perda de sua escolha. Trata-se de um problema de classificação com 3 classes, nesse caso a Cross Entropy é uma função recomendada, que no PyTorch recebe o nome de CrossEntropyLoss:
https://pytorch.org/docs/stable/nn.html#crossentropyloss

**Assim como a rede, as entradas e os rótulos, a função de perda também deve ser carregada na GPU**

In [ ]:
criterion = nn.CrossEntropyLoss().to(device) #cast to GPU

Antes de aplicar a função de perda, vamos fazer o cast dos dados para tensores e extrair as predições y' da rede.

In [ ]:
Xtns = torch.from_numpy(data).float()
Ytns = torch.from_numpy(target)

#cast to GPU

Xtns = Xtns.to(device)
Ytns = Ytns.to(device)

print(Xtns.dtype, Ytns.dtype)

In [ ]:
pred = net(Xtns)

Confira as dimensões de y e y'. Enquanto as predições estão em termos de probabilidade, os rótulos de classificação devem ser valores inteiros referentes aos índices das classes.

In [ ]:
print(pred.shape, Ytns.shape)

print(pred[0].data, Ytns[0].data)

As funções de perda implementadas no PyTorch esperam o seguinte padrão de chamada:

loss = criterion(prediction, target)

Vale lembrar que cada função de perda possui especificidades quanto às dimensões dos seus parâmetros. Para a Cross Entropy:

- prediction: (N, C)
- target(N, )

In [ ]:
loss = criterion(pred, Ytns)
print(loss)

#Regressão

Vamos trabalhar com o dataset de Diabetes, cujo objetivo é prever a progressão da diabetes em um paciente.

https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset


In [ ]:
from sklearn import datasets

diabetes = datasets.load_diabetes()
data = diabetes.data
target = diabetes.target

print(data.shape, target.shape)

print(data[0])
print(target[0])

Implementando o MLP

In [ ]:
class WineClassifier(nn.Module):
  
  def __init__(self, input_size, hidden_size, out_size):
    super(WineClassifier, self).__init__()

    self.hidden = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.out = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax()

  def forward(self, X):

    feature = self.relu(self.hidden(X))
    output = self.softmax(self.out(feature))

    return output
  
input_size   = data.shape[1]
hidden_size  = 32
out_size     = 1 # progressão da diabetes

net = WineClassifier(input_size, hidden_size, out_size).to(device) #cast to GPU


Para solucionar problemas de regressão, as funções de perda correspondentes esperam que ambos o rótulo e a predição tenham a **a mesma dimensionalidade.** Não se trata mais de um problema categórico.

Portando, vamos simular um problema de regressão e aplicar a MSELoss
Documentação: https://pytorch.org/docs/stable/nn.html#mseloss

In [ ]:
criterion = nn.MSELoss().to(device)

# cast na GPU
Xtns = torch.from_numpy(data).float().to(device)
Ytns = torch.from_numpy(target).float().to(device)

print(Xtns.shape, Ytns.shape)

In [ ]:
pred = net(Xtns)
print(pred.shape)

loss = criterion(pred.squeeze(), Ytns)
print(loss.data)

#Documentação

Veja a documentação para consultar a lista de todas as funções de perda implementadas no PyTorch:
https://pytorch.org/docs/stable/nn.html#loss-functions